In [9]:
import lithops
import laspy.copc as lasp
import time

In [10]:
from cloudAPI import CopcHelper

In [11]:
bucket= 'lithops-testing3'
filekey= 'test.copc.laz'
filename= 'lidar.copc.laz'

In [12]:
def helper_test(aux, ibm_cos):
    clock0 = time.time()
    
    #Download header and octree
    helper = CopcHelper(bucket, filekey, filename, ibm_cos)
    helper.load_header()
    helper.load_rootpage()
    
    clock1 = time.time()
    
    #Get random key
    key = helper.get_level_keys(4)[0]
    
    clock2 = time.time()
    
    #Download and read points of a level 0 node
    helper.load_points(helper.get_node(key))
    points = helper.get_points([helper.get_node(key)])
    
    clock3 = time.time()
    
    #Find worker balance
    balance = helper.point_balance(200_000)
    
    clock4 = time.time()
    
    #Download nodes assigned to one worker
    helper.multiple_points_download(balance[0])
    
    clock5 = time.time()

    return clock1 - clock0, clock3 - clock2, str(key), clock4 - clock3, clock5 - clock4, str(balance[0])

In [13]:
if __name__ == "__main__":
    fexec = lithops.LocalhostExecutor()
    fexec.call_async(helper_test, 0)
    results = fexec.get_result()

2022-07-19 11:14:57,149 [INFO] lithops.config -- Lithops v2.6.0
2022-07-19 11:14:57,153 [INFO] lithops.storage.backends.localhost.localhost -- Localhost storage client created
2022-07-19 11:14:57,154 [INFO] lithops.localhost.localhost -- Localhost compute client created
2022-07-19 11:14:57,154 [INFO] lithops.invokers -- ExecutorID 5f2110-1 | JobID A000 - Selected Runtime: python 
2022-07-19 11:14:57,159 [INFO] lithops.invokers -- ExecutorID 5f2110-1 | JobID A000 - Starting function invocation: helper_test() - Total: 1 activations
2022-07-19 11:14:57,195 [INFO] lithops.invokers -- ExecutorID 5f2110-1 | JobID A000 - View execution logs at /tmp/lithops/logs/5f2110-1-A000.log
2022-07-19 11:14:58,151 [INFO] lithops.wait -- ExecutorID 5f2110-1 - Getting results from 1 function activations


    0%|          | 0/1  

2022-07-19 11:15:02,984 [INFO] lithops.executors -- ExecutorID 5f2110-1 - Cleaning temporary data


In [14]:
    print("EXECUTION TIMES OF KEY ", results[2], "seconds")
    print("Download header and octree: ", results[0], "seconds")
    print("Download and read points of a level 4 node: ", results[1], "seconds")

EXECUTION TIMES OF KEY  VoxelKey(level=4, x=0, y=2, z=6) seconds
Download header and octree:  1.2789442539215088 seconds
Download and read points of a level 4 node:  0.678175687789917 seconds


In [15]:
    print("Time to know the nodes assigned to each worker: ", results[3], "seconds")
    print("Time to download the worker nodes: ", results[4], "seconds")    
    print("Downloaded nodes by that worker: ", results[5], "seconds")

Time to know the nodes assigned to each worker:  0.0009152889251708984 seconds
Time to download the worker nodes:  1.6996371746063232 seconds
Downloaded nodes by that worker:  [Entry(key=VoxelKey(level=0, x=0, y=0, z=0), offset=56751611, byte_size=477852, point_count=39213, Entry(key=VoxelKey(level=1, x=0, y=0, z=0), offset=52819705, byte_size=872057, point_count=98085, Entry(key=VoxelKey(level=2, x=0, y=0, z=1), offset=41154681, byte_size=423230, point_count=52333, Entry(key=VoxelKey(level=3, x=0, y=0, z=2), offset=22896782, byte_size=131370, point_count=21876] seconds


In [16]:
    print("Total time a worker would need to start operating: ", results[0] + results[4], "seconds (buffering still needs to be implemented for accurate results)")

Total time a worker would need to start operating:  2.978581428527832 seconds (buffering still needs to be implemented for accurate results)
